In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile
import pandas as pd



In [48]:

masterlist_url = "http://data.gdeltproject.org/gdeltv2/masterfilelist.txt"
save_path = r"C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Arcada-Thesis2\Economic data\GDELT scraper 2"
os.makedirs(save_path, exist_ok=True)

# Fetch the masterfilelist
response = requests.get(masterlist_url)
response.raise_for_status()
lines = response.text.splitlines()
links = [line.split()[-1] for line in lines if line.split()[-1].endswith(".gkg.csv.zip")]
# Reverse the list to start from the end (newest files first)
links = links[::-1]
links = links[:100]  # For testing

# Print files to download
print("Files to download:")
for link in links:
    print(" -", link)

# Download, extract, filter, remove ZIPs
for link in links:
    try:
        zip_path = os.path.join(save_path, os.path.basename(link))

        # Download
        r = requests.get(link, timeout=60)
        r.raise_for_status()
        with open(zip_path, "wb") as f:
            f.write(r.content)

        # Extract CSVs
        with zipfile.ZipFile(zip_path, "r") as z:
            z.extractall(save_path)
            extracted_files = z.namelist()

            for filename in extracted_files:
                if filename.endswith(".csv"):
                    file_path = os.path.join(save_path, filename)
                    df = pd.read_csv(file_path, sep="\t", header=0, low_memory=False)
                    df.columns = [f"column{i+1}" for i in range(df.shape[1])]

                    # Filter by columns 8 and 10
                    df.iloc[:, 9] = df.iloc[:, 9].str.lower()  # column10
                    df = df[df.iloc[:, 9].str.contains("germany|france|italy|spain", case=False, na=False)]
                    df = df[df.iloc[:, 7].notna() & (df.iloc[:, 7] != "")]

                    # Keep only the required columns and rename
                    columns_to_keep = [0, 1, 3, 4, 7, 8, 9, 10, 15, 17]
                    df = df.iloc[:, columns_to_keep]
                    df.columns = ["ID", "Date", "SOURCES", "path", "THEMES", 
                                  "THEMES2", "LOCATIONS", "LOCATIONS2", "TONE", "GCAM"]

                    # Save filtered CSV
                    df.to_csv(file_path, index=False)

        # Delete ZIP after extraction
        os.remove(zip_path)

    except requests.HTTPError as e:
        print(f"HTTP error for {link}: {e}. Skipping.")
    except requests.RequestException as e:
        print(f"Request error for {link}: {e}. Skipping.")
    except zipfile.BadZipFile as e:
        print(f"Bad ZIP file {zip_path}: {e}. Skipping.")
    except Exception as e:
        print(f"Error processing {link}: {e}. Skipping.")


Files to download:
 - http://data.gdeltproject.org/gdeltv2/20251127184500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127183000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127181500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127180000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127174500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127173000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127171500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127170000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127164500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127163000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127161500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127160000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127154500.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/20251127153000.gkg.csv.zip
 - http://data.gdeltproject.org/gdeltv2/202

## OLD script to run chuncks

In [4]:
# Fetch the index page (ignore SSL verification)
response = requests.get(base_url, verify=False)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

# Get only the first 3 .gkg.csv.zip links (ignore gkgcounts)
links = [
    base_url + a["href"]
    for a in soup.find_all("a", href=True)
    if a["href"].endswith(".gkg.csv.zip")
][:1400]

# ✅ Only process elements 231 → 500
links = links[1095:1400]

print("Files to download:")
for link in links:
    print(" -", link)

# Download, extract, filter, and remove ZIPs
for link in links:
    zip_path = os.path.join(save_path, os.path.basename(link))
    print(f"\nDownloading {link} -> {zip_path}")

    r = requests.get(link, verify=False)
    r.raise_for_status()
    with open(zip_path, "wb") as f:
        f.write(r.content)
    print(f"Saved: {zip_path}")

    # Extract CSVs
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(save_path)
        extracted_files = z.namelist()
        print(f"Extracted: {extracted_files}")

        # --- ✅ FILTER HERE ---
        for filename in extracted_files:
            if filename.endswith(".csv"):
                file_path = os.path.join(save_path, filename)
                df = pd.read_csv(file_path, sep="\t", low_memory=False)
                df["LOCATIONS"] = df["LOCATIONS"].str.lower()
                df = df[df["LOCATIONS"].str.contains("germany|france|italy|spain", case=False, na=False)]
                df = df[df["THEMES"].notna() & (df["THEMES"] != "")]
                df.to_csv(file_path, index=False)
                print(f"Filtered and saved: {file_path} ({len(df)} rows kept)")
        # -----------------------

    # Delete ZIP after extraction
    os.remove(zip_path)
    print(f"Deleted ZIP: {zip_path}")

print("\n✅ All .gkg.csv.zip files (231–500) downloaded, filtered, extracted, and cleaned up successfully!")

c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gdeltproject.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Files to download:
 - https://data.gdeltproject.org/gkg/20221022.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221021.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221020.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221019.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221018.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221017.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221016.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221015.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221014.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221013.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221012.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221011.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221010.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221009.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221008.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221007.gkg.csv.zip
 - https://data.gdeltproject.org/gkg/20221006.gkg.csv

c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gdeltproject.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20221022.gkg.csv.zip
Extracted: ['20221022.gkg.csv']
Filtered and saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20221022.gkg.csv (3236 rows kept)
Deleted ZIP: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20221022.gkg.csv.zip



c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gdeltproject.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211224.gkg.csv.zip
Extracted: ['20211224.gkg.csv']
Filtered and saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211224.gkg.csv (4534 rows kept)
Deleted ZIP: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211224.gkg.csv.zip



c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gdeltproject.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211223.gkg.csv.zip
Extracted: ['20211223.gkg.csv']
Filtered and saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211223.gkg.csv (5660 rows kept)
Deleted ZIP: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211223.gkg.csv.zip



c:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.gdeltproject.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211222.gkg.csv.zip
Extracted: ['20211222.gkg.csv']
Filtered and saved: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211222.gkg.csv (6440 rows kept)
Deleted ZIP: C:\Users\Ricks\OneDrive\Skrivbord\Arcada\Thesis\Economic data\GDELT scraping\20211222.gkg.csv.zip

✅ All .gkg.csv.zip files (231–500) downloaded, filtered, extracted, and cleaned up successfully!
